In [1]:
import numpy as np
#from lhereader import LHEReader

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

### extract info for LQ paper

### BACKGROUNDS

#### tt

In [4]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/tt/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-tt/'


ev = []


#for iev in range(1,526):
for iev in range(527,551):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 527, '   # events ok so far: ', 1514)
('run: ', 528, '   # events ok so far: ', 3101)
('run: ', 529, '   # events ok so far: ', 4632)
('run: ', 530, '   # events ok so far: ', 6177)
('run: ', 531, '   # events ok so far: ', 7726)
('run: ', 532, '   # events ok so far: ', 9297)
('run: ', 533, '   # events ok so far: ', 10859)
('run: ', 534, '   # events ok so far: ', 12403)
('run: ', 535, '   # events ok so far: ', 13945)
('run: ', 536, '   # events ok so far: ', 15481)
('run: ', 537, '   # events ok so far: ', 16965)
('run: ', 538, '   # events ok so far: ', 18547)
('run: ', 539, '   # events ok so far: ', 20104)
('run: ', 540, '   # events ok so far: ', 21683)
('run: ', 541, '   # events ok so far: ', 23310)
('run: ', 542, '   # events ok so far: ', 24877)
('run: ', 543, '   # events ok so far: ', 26422)
('run: ', 544, '   # events ok so far: ', 28021)
('run: ', 545, '   # events ok so far: ', 29637)
('run: ', 546, '   # events ok so far: ', 31236)
('run: ', 547, '   # event

In [ ]:
# from run 1 to 525,  788570
# from run 527 to 550,   37452

# total: 788570+37452 = 826022

In [5]:
788570+37452

826022

#### fake tt

In [6]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/fakett/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-fakett/'


ev = []



for iev in range(1,126):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 0)
('run: ', 2, '   # events ok so far: ', 0)
('run: ', 3, '   # events ok so far: ', 0)
('run: ', 4, '   # events ok so far: ', 0)
('run: ', 5, '   # events ok so far: ', 0)
('run: ', 6, '   # events ok so far: ', 0)
('run: ', 7, '   # events ok so far: ', 0)
('run: ', 8, '   # events ok so far: ', 1)
('run: ', 9, '   # events ok so far: ', 1)
('run: ', 10, '   # events ok so far: ', 1)
('run: ', 11, '   # events ok so far: ', 1)
('run: ', 12, '   # events ok so far: ', 1)
('run: ', 13, '   # events ok so far: ', 1)
('run: ', 14, '   # events ok so far: ', 1)
('run: ', 15, '   # events ok so far: ', 1)
('run: ', 16, '   # events ok so far: ', 1)
('run: ', 17, '   # events ok so far: ', 1)
('run: ', 18, '   # events ok so far: ', 1)
('run: ', 19, '   # events ok so far: ', 1)
('run: ', 20, '   # events ok so far: ', 1)
('run: ', 21, '   # events ok so far: ', 1)
('run: ', 22, '   # events ok so far: ', 1)
('run: ', 23, '   # events ok so far: ', 

In [ ]:
# total: 8

#### single t part 1 (part of single t)

In [7]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/singletP1/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-singletP1/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 260)
('run: ', 2, '   # events ok so far: ', 531)
('run: ', 3, '   # events ok so far: ', 783)
('run: ', 4, '   # events ok so far: ', 1037)
('run: ', 5, '   # events ok so far: ', 1283)
('run: ', 6, '   # events ok so far: ', 1528)
('run: ', 7, '   # events ok so far: ', 1770)
('run: ', 8, '   # events ok so far: ', 2040)
('run: ', 9, '   # events ok so far: ', 2276)
('run: ', 10, '   # events ok so far: ', 2518)
('run: ', 11, '   # events ok so far: ', 2769)
('run: ', 12, '   # events ok so far: ', 3032)
('run: ', 13, '   # events ok so far: ', 3286)
('run: ', 14, '   # events ok so far: ', 3532)
('run: ', 15, '   # events ok so far: ', 3777)
('run: ', 16, '   # events ok so far: ', 4027)
('run: ', 17, '   # events ok so far: ', 4299)
('run: ', 18, '   # events ok so far: ', 4573)
('run: ', 19, '   # events ok so far: ', 4847)
('run: ', 20, '   # events ok so far: ', 5079)
('run: ', 21, '   # events ok so far: ', 5316)
('run: ', 22, '   # event

('run: ', 171, '   # events ok so far: ', 147147)
('run: ', 172, '   # events ok so far: ', 148265)
('run: ', 173, '   # events ok so far: ', 149409)
('run: ', 174, '   # events ok so far: ', 150548)
('run: ', 175, '   # events ok so far: ', 151668)
('run: ', 176, '   # events ok so far: ', 152796)
('run: ', 177, '   # events ok so far: ', 153945)
('run: ', 178, '   # events ok so far: ', 155097)
('run: ', 179, '   # events ok so far: ', 156186)
('run: ', 180, '   # events ok so far: ', 157328)
('run: ', 181, '   # events ok so far: ', 158471)
('run: ', 182, '   # events ok so far: ', 159547)
('run: ', 183, '   # events ok so far: ', 160614)
('run: ', 184, '   # events ok so far: ', 161694)
('run: ', 185, '   # events ok so far: ', 162847)
('run: ', 186, '   # events ok so far: ', 163929)
('run: ', 187, '   # events ok so far: ', 165079)
('run: ', 188, '   # events ok so far: ', 166216)
('run: ', 189, '   # events ok so far: ', 167357)
('run: ', 190, '   # events ok so far: ', 168494)


In [ ]:
# total: 179677

In [9]:
1100*200

220000

#### single t comp (part of single t)

In [10]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/singlet-comp/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-singlet-comp/'


ev = []



for iev in range(1,101):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 930)
('run: ', 2, '   # events ok so far: ', 1871)
('run: ', 3, '   # events ok so far: ', 2757)
('run: ', 4, '   # events ok so far: ', 3646)
('run: ', 5, '   # events ok so far: ', 4586)
('run: ', 6, '   # events ok so far: ', 5528)
('run: ', 7, '   # events ok so far: ', 6478)
('run: ', 8, '   # events ok so far: ', 7444)
('run: ', 9, '   # events ok so far: ', 8373)
('run: ', 10, '   # events ok so far: ', 9306)
('run: ', 11, '   # events ok so far: ', 10269)
('run: ', 12, '   # events ok so far: ', 11203)
('run: ', 13, '   # events ok so far: ', 12187)
('run: ', 14, '   # events ok so far: ', 13133)
('run: ', 15, '   # events ok so far: ', 14073)
('run: ', 16, '   # events ok so far: ', 14984)
('run: ', 17, '   # events ok so far: ', 15923)
('run: ', 18, '   # events ok so far: ', 16884)
('run: ', 19, '   # events ok so far: ', 17813)
('run: ', 20, '   # events ok so far: ', 18774)
('run: ', 21, '   # events ok so far: ', 19712)
('run: ', 22

IOError: File does not exist: /media/andres/Elements/Leptoquarks/backgrounds/singlet-comp/LO/Auto_Results_LHCO/run_0101.lhco

In [ ]:
# total: 94219

In [11]:
930*100

93000

#### W plus b b jets

In [2]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/Wbbjets/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-Wbbjets/'


ev = []



for iev in range(1,151):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

/home/andres/anaconda3/envs/py2/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


('run: ', 1, '   # events ok so far: ', 1076)
('run: ', 2, '   # events ok so far: ', 2148)
('run: ', 3, '   # events ok so far: ', 3227)
('run: ', 4, '   # events ok so far: ', 4310)
('run: ', 5, '   # events ok so far: ', 5399)
('run: ', 6, '   # events ok so far: ', 6444)
('run: ', 7, '   # events ok so far: ', 7540)
('run: ', 8, '   # events ok so far: ', 8603)
('run: ', 9, '   # events ok so far: ', 9655)
('run: ', 10, '   # events ok so far: ', 10700)
('run: ', 11, '   # events ok so far: ', 11796)
('run: ', 12, '   # events ok so far: ', 12898)
('run: ', 13, '   # events ok so far: ', 14010)
('run: ', 14, '   # events ok so far: ', 15104)
('run: ', 15, '   # events ok so far: ', 16163)
('run: ', 16, '   # events ok so far: ', 17302)
('run: ', 17, '   # events ok so far: ', 18344)
('run: ', 18, '   # events ok so far: ', 19439)
('run: ', 19, '   # events ok so far: ', 20551)
('run: ', 20, '   # events ok so far: ', 21601)
('run: ', 21, '   # events ok so far: ', 22637)
('run: ', 

In [ ]:
# total: 162448

In [3]:
1076*150

161400

#### Z plus b b jets

In [4]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/Zbbjets/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-Zbbjets/'


ev = []



for iev in range(1,31):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 111)
('run: ', 2, '   # events ok so far: ', 245)
('run: ', 3, '   # events ok so far: ', 362)
('run: ', 4, '   # events ok so far: ', 492)
('run: ', 5, '   # events ok so far: ', 612)
('run: ', 6, '   # events ok so far: ', 730)
('run: ', 7, '   # events ok so far: ', 846)
('run: ', 8, '   # events ok so far: ', 955)
('run: ', 9, '   # events ok so far: ', 1089)
('run: ', 10, '   # events ok so far: ', 1210)
('run: ', 11, '   # events ok so far: ', 1325)
('run: ', 12, '   # events ok so far: ', 1433)
('run: ', 13, '   # events ok so far: ', 1544)
('run: ', 14, '   # events ok so far: ', 1668)
('run: ', 15, '   # events ok so far: ', 1794)
('run: ', 16, '   # events ok so far: ', 1919)
('run: ', 17, '   # events ok so far: ', 2050)
('run: ', 18, '   # events ok so far: ', 2185)
('run: ', 19, '   # events ok so far: ', 2307)
('run: ', 20, '   # events ok so far: ', 2419)
('run: ', 21, '   # events ok so far: ', 2524)
('run: ', 22, '   # events ok 

In [ ]:
# total: 3662

In [5]:
111*30

3330

#### ttW (part of ttV)

In [6]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/ttW/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-ttW/'


ev = []



for iev in range(1,52):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2823)
('run: ', 2, '   # events ok so far: ', 5661)
('run: ', 3, '   # events ok so far: ', 8473)
('run: ', 4, '   # events ok so far: ', 11264)
('run: ', 5, '   # events ok so far: ', 14013)
('run: ', 6, '   # events ok so far: ', 16860)
('run: ', 7, '   # events ok so far: ', 19587)
('run: ', 8, '   # events ok so far: ', 22362)
('run: ', 9, '   # events ok so far: ', 25093)
('run: ', 10, '   # events ok so far: ', 27918)
('run: ', 11, '   # events ok so far: ', 30657)
('run: ', 12, '   # events ok so far: ', 33493)
('run: ', 13, '   # events ok so far: ', 36273)
('run: ', 14, '   # events ok so far: ', 38909)
('run: ', 15, '   # events ok so far: ', 41721)
('run: ', 16, '   # events ok so far: ', 44580)
('run: ', 17, '   # events ok so far: ', 47309)
('run: ', 18, '   # events ok so far: ', 50085)
('run: ', 19, '   # events ok so far: ', 52775)
('run: ', 20, '   # events ok so far: ', 55625)
('run: ', 21, '   # events ok so far: ', 58455)
('ru

In [ ]:
# total: 141919

In [7]:
2823*51

143973

#### ttZ (part of ttV)

In [8]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/ttZ/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-ttZ/'


ev = []



for iev in range(1,26):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 659)
('run: ', 2, '   # events ok so far: ', 1319)
('run: ', 3, '   # events ok so far: ', 1974)
('run: ', 4, '   # events ok so far: ', 2611)
('run: ', 5, '   # events ok so far: ', 3306)
('run: ', 6, '   # events ok so far: ', 3956)
('run: ', 7, '   # events ok so far: ', 4620)
('run: ', 8, '   # events ok so far: ', 5229)
('run: ', 9, '   # events ok so far: ', 5873)
('run: ', 10, '   # events ok so far: ', 6514)
('run: ', 11, '   # events ok so far: ', 7200)
('run: ', 12, '   # events ok so far: ', 7841)
('run: ', 13, '   # events ok so far: ', 8423)
('run: ', 14, '   # events ok so far: ', 9084)
('run: ', 15, '   # events ok so far: ', 9743)
('run: ', 16, '   # events ok so far: ', 10382)
('run: ', 17, '   # events ok so far: ', 11046)
('run: ', 18, '   # events ok so far: ', 11706)
('run: ', 19, '   # events ok so far: ', 12338)
('run: ', 20, '   # events ok so far: ', 12973)
('run: ', 21, '   # events ok so far: ', 13614)
('run: ', 22, '  

In [ ]:
# total: 16178

In [9]:
659*25

16475

#### ttH

In [10]:
Folder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/ttH/LO/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/backgrounds/DATA/dats-ttH/'


ev = []



for iev in range(1,26):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 480)
('run: ', 2, '   # events ok so far: ', 979)
('run: ', 3, '   # events ok so far: ', 1466)
('run: ', 4, '   # events ok so far: ', 1961)
('run: ', 5, '   # events ok so far: ', 2448)
('run: ', 6, '   # events ok so far: ', 2924)
('run: ', 7, '   # events ok so far: ', 3420)
('run: ', 8, '   # events ok so far: ', 3894)
('run: ', 9, '   # events ok so far: ', 4361)
('run: ', 10, '   # events ok so far: ', 4864)
('run: ', 11, '   # events ok so far: ', 5336)
('run: ', 12, '   # events ok so far: ', 5860)
('run: ', 13, '   # events ok so far: ', 6329)
('run: ', 14, '   # events ok so far: ', 6807)
('run: ', 15, '   # events ok so far: ', 7304)
('run: ', 16, '   # events ok so far: ', 7782)
('run: ', 17, '   # events ok so far: ', 8301)
('run: ', 18, '   # events ok so far: ', 8775)
('run: ', 19, '   # events ok so far: ', 9266)
('run: ', 20, '   # events ok so far: ', 9760)
('run: ', 21, '   # events ok so far: ', 10247)
('run: ', 22, '   # eve

In [ ]:
# total: 12068

In [11]:
480*25

12000